In [5]:
import pandas as pd
from models import TransactionModel, SplitModel
from utils.helpers import generate_unique_id
from datetime import datetime
from decimal import Decimal, ROUND_HALF_UP

In [6]:
df = pd.read_csv('Finances/Finances - Joint.csv')
df.columns = df.iloc[0]
df = df[1:319]
df.reset_index(drop=True, inplace=True)
df.head()

,Date,Description,Category,Amount (USD),Austin,Sara,Account Paid To,Notes
0,7/10/2025,Arizona DMV License,OTHER,$12.00,$12.00,$0.00,USBANK,NaN
1,7/10/2025,Walmart,HOUSEHOLD,$15.89,$15.89,$0.00,USBANK,Buckets and rags
2,7/11/2025,IKEA Cover and Bench,HOUSEHOLD,$97.27,$64.84,$32.43,USBANK,NaN
3,7/11/2025,Frys,HOUSEHOLD,$9.77,$9.77,$0.00,USBANK,Lock for uhaul
4,7/11/2025,UHaul,OTHER,$232.29,$232.29,$0.00,USBANK,Moving right along


In [7]:
transactionModel = TransactionModel()
splitModel = SplitModel()

In [8]:
for index, row in df.iterrows():
    match (row['Account Paid To']):
        case "USBANK":
            bank_account_id = 1
        case "SAPPHIRE":
            bank_account_id = 2
        case "BOFA":
            bank_account_id = 3
        case "JOINT CHASE":
            bank_account_id = 4
        case "DISCOVER":
            bank_account_id = 5

    if row['Amount (USD)'].startswith('-'):
        transactionType='income'
        amount = row['Amount (USD)'][2:].replace(",", "")
        austinSpent = row['Austin'][2:].replace(",", "")
        saraSpent = row['Sara'][2:].replace(",", "")
    else:
        transactionType='expense'
        amount = row['Amount (USD)'][1:].replace(",", "")
        austinSpent = row['Austin'][1:].replace(",", "")
        saraSpent = row['Sara'][1:].replace(",", "")

    transaction = {
        'item_name': 'Unknown',
        'category': row['Category'],
        'store': row['Description'],
        'date': datetime.strptime(row['Date'], "%m/%d/%Y").strftime("%Y-%m-%#d"),
        'price': amount,
        'user_id': 2,
        'bank_account_id': bank_account_id,
        'type': transactionType,
        'receipt_image': '',
        'receipt_json': '',
        'group_id': 1
    }

    if row['Austin'] != "$0.00" and row['Sara'] != "$0.00":
        receipt_group_id = generate_unique_id()
        transaction['receipt_group_id'] = receipt_group_id

        percentage_austin = (
            (Decimal(austinSpent) / (Decimal(austinSpent) + Decimal(saraSpent))) * 100
            ).quantize(Decimal("0.01"), ROUND_HALF_UP)
        percentage_sara = 100 - percentage_austin
        split1 = {
            'receipt_group_id': receipt_group_id,
            'user_id': 2,
            'amount': austinSpent,
            'percentage': percentage_austin
        }

        split2 = {
            'receipt_group_id': receipt_group_id,
            'user_id': 3,
            'amount': saraSpent,
            'percentage': percentage_sara
        }

        splitModel.create(split1)
        splitModel.create(split2)
    else:
        transaction['receipt_group_id'] = ""

    transactionModel.create(transaction)

    break


For Income

In [ ]:
df = pd.read_csv('Finances - Sheet7.csv')
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
for index, row in df.iterrows():
    transactionType='income'
    amount = row['Amount (USD)'][1:].replace(",", "")
    if row['Person'] == "Austin":
        user_id = 2
        bank_account_id = 7
    else:
        user_id = 3
        bank_account_id = 6

    transaction = {
        'item_name': 'Income',
        'category': row['Category'],
        'store': row['Description'],
        'date': datetime.strptime(row['Date'], "%m/%d/%Y").strftime("%Y-%m-%#d"),
        'price': amount,
        'user_id': user_id,
        'bank_account_id': bank_account_id,
        'type': "income",
        'receipt_image': '',
        'group_id': '1',
        'receipt_group_id': "",
    }

    transactionModel.create(transaction)